In [ ]:
import pandas as pd
import re

# Load the CSV file
data = pd.read_csv('../../../dataset/train.csv')

# Filter to only include item_weight entity
item_weight_data = data[data['entity_name'] == 'height']

# Extract numeric value and unit from entity_value
def split_value(value):
    match = re.match(r'(\d+(\.\d+)?)(\s+)?(yard|inch|meter|millimeter|foot|centimeter)', value)
    if match:
        number = float(match.group(1))
        unit = match.group(4)
        return number, unit
    return None, None

item_weight_data[['numeric_value', 'unit']] = item_weight_data['entity_value'].apply(split_value).apply(pd.Series)

# Drop rows with missing values
item_weight_data.dropna(inplace=True)

# Save processed data to a new CSV file
item_weight_data.to_csv('height.csv', index=True)

print("Data preprocessing complete. Processed data saved to 'processed_item_weight_data.csv'.")

In [2]:
from utils_height import download_images

In [ ]:
download_images(item_weight_data['image_link'].tolist(),  item_weight_data.index.tolist(), "dataset")

In [14]:
import os
import logging
import pandas as pd
from paddleocr import PaddleOCR
from tqdm import tqdm

logging.getLogger('ppocr').setLevel(logging.ERROR)

image_folder = 'dataset/'  # Adjust this to the correct folder containing your images
ocr = PaddleOCR(use_angle_cls=True, lang='en')  # Initialize PaddleOCR
def extract_ocr_text(index):
    image_path = os.path.join("datasets", f"{index}.jpg")
    if not os.path.exists(image_path):
        return ""
    
    result = ocr.ocr(image_path)
    ocr_text = ""
    
    # Extract text from OCR results
    for line in result:
        for box, (text, confidence) in line:
            ocr_text += text + " "
    
    return index, ocr_text.strip()

In [15]:
# Assuming `data` is your DataFrame
data = pd.read_csv("height.csv")
indices = data['id'].to_list()
ocr_results = [extract_ocr_text(index) for index in tqdm(indices, desc="Extracting OCR")]

# Add the OCR results to the DataFrame
ocr_text_dict = dict(ocr_results)
data['ocr_text'] = data['id'].map(ocr_text_dict)

# Save the updated DataFrame with the OCR text to a new CSV
updated_csv_file = 'height_with_ocr.csv'
data.to_csv(updated_csv_file, index=True)

print(f"OCR data has been added to the CSV. Updated file saved as '{updated_csv_file}'.")

TypeError: 'module' object is not callable